## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 데이터

### 데이터 가져오기

In [6]:
origin_data = pd.read_excel('./3차_VOD_2308.xlsx', index_col = None)

### 데이터 전처리

In [14]:
# 불필요한 컬럼 제거
drop_data = origin_data.drop(['Unnamed: 0', 'ct_cl', 'genre_of_ct_cl', 'use_tms', 'SMRY', 'disp_rtm', 'strt_dt'], axis = 1)

In [29]:
# 데이터에서 불필요한 부분 제거

import re

# 제목 데이터에서 불필요한 부분을 제거하고
# 전처리된 제목을 새로운 컬럼으로 추가하는 함수
def get_rid_asset(dataframe):
    # 괄호 안의 불필요한 부분 패턴
    pattern = re.compile('\([A-Za-z가-힣\s]+\)|\([가-힣\d\s]+\)|\([\d\/]{5,}\)')

    new_data = []

    for item in dataframe['asset_nm']:
        name = str(item)
        match_pattern = pattern.findall(name)

        for dt in match_pattern:
            name = name.replace(dt, "")
            #new_data.append(dt)

        new_data.append(name) 
    
    # 제목과 함께 존재하는 회차 정보 제거
    pattern = re.compile('\s[\d]+회')

    series = []

    for item in new_data:
        name = str(item)

        # 정규 표현식을 만족하는 경우 - 드라마 컨텐츠
        try:
            match_pattern = pattern.findall(name)

            for dt in match_pattern:
                name = name.replace(dt, "")
            series.append(name)

        # 정규 표현식을 만족하지 않는 경우 - 영화(~~회 가 없음)
        except:
            series.append(item)
    
    # 데이터 프레임에 처리 결과 컬럼을 추가
    dataframe['series_name'] = series

In [30]:
# 데이터에 함수를 적용
get_rid_asset(drop_data)

# 함수 적용 결과 확인
#print(drop_data.head())

In [35]:
# 출연진 정보가 없는 데이터 제거
no_actr_data = drop_data.drop(drop_data[drop_data['ACTR_DISP'] == '-'].index, axis = 0)

# 제거 확인 - 출력되는 데이터 없음
# print(no_actr_data[no_actr_data['ACTR_DISP'] == '-'])

In [ ]:
drop_data.head()

In [43]:
# 특정 subsr 의 시청 데이터에서 출연진 가져오기
view_data = no_actr_data[drop_data['subsr'] == 65941000]

# 데이터 확인
print(view_data.head())

       subsr                     asset_nm            ACTR_DISP  series_name
0   65941000  (HD)그것이알고싶다 1361회(23/07/22)                  김상중      그것이알고싶다
12  65941000     (HD)인간극장 3305회(23/05/29)            이용철,스즈키 준         인간극장
20  65941000    과학수사대 스모킹 건 12회(23/06/14)  안현모,이유리,유성호,김복준,표창원  과학수사대 스모킹 건
35  65941000     (HD)인간극장 3325회(23/06/26)                  강윤아         인간극장
81  65941000     (HD)인간극장 3308회(23/06/02)            이용철,스즈키 준         인간극장


C:\Users\USER\AppData\Local\Temp\ipykernel_13544\2122190420.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  view_data = no_actr_data[drop_data['subsr'] == 65941000]


## 출연진 기반 추천

### 출연진 데이터

In [ ]:
# 출연진 정보를 저장할 list
actr_list = []

# 시청 데이터를 순회하면서 출연진 정보를 저장
for _, item in view_data.iterrows():
    if ',' in item['ACTR_DISP']:
        item_list = item['ACTR_DISP'].split(',')
    else:
        item_list = [item['ACTR_DISP']]
    
    for actr in item_list:
        actr_list.append(actr)

# 확인
print(actr_list)

In [55]:
# counter 를 사용해서 출연 횟수 확인
from collections import Counter

count = Counter(actr_list)

# 가장 많이 출연한 상위 3위 출연진 정보
best3_actr = count.most_common(3)

In [67]:
# 상위 3위 출연진에 대해 가중치 설정
score_sum = 0
for actr_score in best3_actr:
    score_sum += actr_score[1]

w_best3 = []
for actr_score in best3_actr:
    w_best3.append([actr_score[0], round((actr_score[1] / score_sum), 2)])
    
print(w_best3)

[['이용철', 0.36], ['스즈키 준', 0.36], ['안현모', 0.29]]


In [ ]:
actr_score_list = []

# 각 행을 순회하면서 출연진 정보에 대한 점수를 더해서 데이터 생성
for _, item in drop_data.iterrows():
    score_data = 0
    
    for name_score in w_best3:
        if name_score[0] in str(item['ACTR_DISP']):
            score_data += name_score[1]
    actr_score_list.append(score_data)
    
# 기존 데이터에 컬럼 추가
drop_data['ATCR_SCORE'] = actr_score_list

# 확인
drop_data.head()

In [80]:
# 출연진 점수를 기준으로 순위 부여
drop_data['rank'] = drop_data['ATCR_SCORE'].rank(method = 'min', na_option = 'bottom',
                                                    ascending = False)
# 확인
drop_data['ATCR_SCORE'].value_counts()

0.00    5579
0.29      31
0.72       5
Name: ATCR_SCORE, dtype: int64